In [1]:
import sqlite3
from sqlite3 import Error

# Create db_file.db, close

In [56]:
def create_connection(db_file):
    try:
        conn=sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e) #show any sqlite error messages
    finally:
        conn.close()

In [57]:
"""create pythonsqlite.db, close"""
create_connection("pythonsqlite.db")

2.6.0


# Create .db in memory, close

In [24]:
def create_connection_mem():
    """create/open .db in memory, close"""
    try:
        conn=sqlite3.connect(':memory:')
        print(sqlite3.version)
    except Error as e:
        print(e) #show any sqlite error messages
    finally:
        conn.close()

In [25]:
"""create .db in memory, close"""
create_connection_mem()

2.6.0


# Create db_file.db

In [14]:
def open_connection(db_file):
    """create/open db_file.db"""
    try:
        conn=sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e) #show any sqlite error messages
    return None

In [93]:
"""open/create pythonsqlite.db"""
conn=open_connection("/Users/lickivy/Desktop/SQLite/sqlitetutorial/pythonsqlite.db")

In [94]:
conn

In [95]:
"""can't do it without conn= """
open_connection("/Users/lickivy/Desktop/SQLite/sqlitetutorial/pythonsqlite.db")
#current connection is:

In [96]:
conn
#definition conn is:

In [97]:
"""open/create pythonsqlite.db"""
conn=open_connection("/Users/lickivy/Desktop/SQLite/sqlitetutorial/pythonsqlite.db")

In [98]:
conn
#current connection = definition conn is:

# Create table

In [66]:
def create_table(conn, create_table_sql):
    """create table in conn"""
    try:
        c=conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e) #show any sqlite error messages

In [70]:
"""can't do it shorter way"""
conn.cursor.execute(sql_create_projects_table)

AttributeError: 'builtin_function_or_method' object has no attribute 'execute'

In [99]:
"""create tables projects, tasks in pythonsqlite.db"""

sql_create_projects_table="""create table if not exists
    projects(
    id integer primary key,
    name text not null,
    begin_date text,
    end_date text);"""
sql_create_tasks_table="""create table if not exists
     tasks(
     id integer primary key,
     name text not null,
     priority integer,
     status_id integer not null,
     project_id integer not null,
     begin_date text not null,
     end_date text not null,
     foreign key(project_id) references projects(id));"""

if conn is not None:
    #create projects table
    create_table(conn,sql_create_projects_table)
    #create tasks table
    create_table(conn,sql_create_tasks_table)
else:
    print("Error: cannot create the database connection")

# Insert row, return rowid

In [100]:
def create_project(conn,row):
    """Insert row into projects table, return rowid"""
    sql="""insert into projects(name,begin_date,end_date)
    values(?,?,?)"""
    c=conn.cursor()
    c.execute(sql,row)
    return c.lastrowid

def create_task(conn,row):
    """Insert row into tasks table, return rowid"""
    sql="""insert into tasks(name,priority,status_id,
    project_id,begin_date,end_date) 
    values(?,?,?,?,?,?)"""
    c=conn.cursor()
    c.execute(sql,row)
    return c.lastrowid

In [104]:
"""insert row1 into projects; row2, row3 into tasks"""
with conn:

    row1=('Cool App with SQLite & Python','2015-01-01','2015-01-30');
    #insert row1
    project_rowid=create_project(conn,row1)
    
    row2=('Analyze the requirements of the app',1,1,project_rowid,
         '2015-01-01','2015-01-02')
    row3=('Confirm with user about the top requirements',1,1,
         project_rowid,'2015-01-03','2015-01-05')
    #insert row2, row3
    create_task(conn,row2)
    create_task(conn,row3)

# Delete row by rowid

In [108]:
def delete_task(conn,id):
    """delete row from tasks by rowid"""
    sql="delete from tasks where rowid=?"
    c=conn.cursor()
    c.execute(sql,(id,)) #must use (id,) else unsupported type

In [109]:
"""delete row 2 from tasks"""
with conn:
    delete_task(conn,2);

# Delete all rows

In [110]:
def delete_all_tasks(conn):
    """delete all rows from tasks table"""
    sql='delete from tasks'
    c=conn.cursor()
    c.execute(sql)

In [111]:
"""delete all rows from tasks"""
with conn:
    delete_all_tasks(conn);

# Update data by rowid

In [113]:
def update_task(conn,update):
    """update priority, begin_date, and end date in tasks table"""
    sql='''update tasks
    set priority=?,
    begin_date=?,
    end_date=?
    where rowid=?'''
    c=conn.cursor()
    c.execute(sql,update)

In [114]:
"""update priority, begin_date, and end date of tasks table row 2"""

#recreate tasks table
create_task(conn,row2)
create_task(conn,row3)

#update
with conn:
    update_task(conn,(2,'2015-01-04','2015-01-06',2))

# Select 

In [117]:
def select_all_tasks(conn):
    """select * from tasks"""
    c=conn.cursor()
    c.execute("select * from tasks")
    output=c.fetchall()
    for row in output:
        print(row)

In [118]:
"""select * from tasks"""
with conn:
    select_all_tasks(conn)

(1, 'Analyze the requirements of the app', 1, 1, 1, '2015-01-01', '2015-01-02')
(2, 'Confirm with user about the top requirements', 2, 1, 1, '2015-01-04', '2015-01-06')


# Select where

In [126]:
def select_task_by_priority(conn,priority):
    """select from tasks by priority"""
    c=conn.cursor()
    c.execute("select priority,begin_date from tasks where priority=?",(priority,))
    output=c.fetchall()
    print(output)

In [127]:
"""select from tasks where priority=1"""
select_task_by_priority(conn,1)

[(1, '2015-01-01')]


# Can combine c=conn.cursor(), c.execute(sql,inputs), print(c.fetchall()) to do anything

In [134]:
sql="select * from tasks inner join projects on tasks.project_id=projects.rowid"
c=conn.cursor()
c.execute(sql)
output=c.fetchall()
for row in output:
    print(row)

(1, 'Analyze the requirements of the app', 1, 1, 1, '2015-01-01', '2015-01-02', 1, 'Cool App with SQLite & Python', '2015-01-01', '2015-01-30')
(2, 'Confirm with user about the top requirements', 2, 1, 1, '2015-01-04', '2015-01-06', 1, 'Cool App with SQLite & Python', '2015-01-01', '2015-01-30')


In [135]:
"""check connection"""
conn

In [136]:
"""close connection"""
conn.close()